In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls

    return cls.__name__.split('.')[-1]


def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in (type(None), int):
        return repr(obj)
    else:
        return f'<{cls_name(obj)} object>'


def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print(f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')


class Overriding:
    def __get__(self, obj, objtype):
        print_args('get', self, obj, objtype)

    def __set__(self, obj, value):
        print_args('set', self, obj, value)


class OverridingNoGet:
    def __set__(self, obj, value):
        print_args('set', self, obj, value)


class NonOverriding:
    def __get__(self, obj, objtype):
        print_args('get', self, obj, objtype)


class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))


"""
To watch overriding descriptor

    >>> obj = Managed()
    >>> obj.over
    -> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
    >>> Managed.over
    -> Overriding.__get__(<Overriding object>, None, <class Managed>)
    >>> obj.over = 7
    -> Overriding.__set__(<Overriding object>, <Managed object>, 7)
    >>> obj.over
    -> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
    >>> obj.__dict__['over'] = 8
    >>> vars(obj)
    {'over': 8}
    >>> obj.over
    -> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
    >>> getattr(obj, 'over')
    -> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
    >>> obj.__dict__['over']
    8

Overriding descriptor without __get__()

    >>> obj.over_no_get  # doctest: +ELLIPSIS
    <__main__.OverridingNoGet object at ...>
    >>> Managed.over_no_get  # doctest: +ELLIPSIS
    <__main__.OverridingNoGet object at ...>
    >>> obj.over_no_get = 7
    -> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
    >>> obj.over_no_get  # doctest: +ELLIPSIS
    <__main__.OverridingNoGet object at ...>
    >>> obj.__dict__['over_no_get'] = 9
    >>> obj.over_no_get
    9
    >>> obj.over_no_get = 7
    -> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
    >>> obj.over_no_get
    9

Non-overriding descriptor

    >>> obj.non_over
    -> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)
    >>> obj.non_over = 7
    >>> obj.non_over
    7
    >>> Managed.non_over
    -> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)
    >>> del obj.non_over
    >>> obj.non_over
    -> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)

Disabling descriptor

    >>> Managed.over = 1
    >>> Managed.over_no_get = 2
    >>> Managed.non_over = 3
    >>> obj = Managed()
    >>> obj.over, obj.over_no_get, obj.non_over
    (1, 2, 3)

Method is non-overriding descriptor

    >>> obj = Managed()
    >>> obj.spam  # doctest: +ELLIPSIS
    <bound method Managed.spam of <__main__.Managed object at ...>>
    >>> Managed.spam  # doctest: +ELLIPSIS
    <function Managed.spam at ...>
    >>> obj.spam = 7
    >>> obj.spam
    7

"""  # noqa: E501

doctest.testmod()

TestResults(failed=0, attempted=34)

In [4]:
import collections


class Text(collections.UserString):
    def __repr__(self):
        return f'Text({self.data!r})'

    def reverse(self):
        return self[::-1]


"""

>>> word = Text('forward')
>>> word
Text('forward')
>>> word.reverse()
Text('drawrof')
>>> Text.reverse(Text('backward'))
Text('drawkcab')
>>> type(Text.reverse), type(word.reverse)
(<class 'function'>, <class 'method'>)
>>> list(map(Text.reverse, ['rapaid', (10, 20, 30), Text('stressed')]))
['diapar', (30, 20, 10), Text('desserts')]
>>> Text.reverse.__get__(word)
<bound method Text.reverse of Text('forward')>
>>> Text.reverse.__get__(None, Text)  # doctest: +ELLIPSIS
<function Text.reverse at ...>
>>> word.reverse
<bound method Text.reverse of Text('forward')>
>>> word.reverse.__self__
Text('forward')
>>> word.reverse.__func__ is Text.reverse
True
"""

doctest.testmod()

TestResults(failed=0, attempted=11)